pca_glove_200features 

In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [2]:
import warnings
warnings.filterwarnings("ignore") 

In [3]:
import os
import numpy as np 
import pandas as pd
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

In [4]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_Glove/data.csv"
df = pd.read_csv(path)
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [5]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [6]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [7]:
#define bins
bins = [0,2,4,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names)

In [8]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [9]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [10]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [11]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_Glove/glove_withstopwordchangelevels_200_features.csv"
x = pd.read_csv(path)
x.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,190,191,192,193,194,195,196,197,198,199
0,0,-0.538296,-5.466138,-0.540832,-1.155411,1.677357,-3.437855,-0.523041,-2.208639,0.969747,...,0.255608,0.275158,-0.185373,0.103836,0.040434,-0.059653,-0.189996,0.364697,-0.137938,-0.160443
1,1,-2.542841,4.907548,1.447379,4.933750,-4.479990,0.505414,4.011756,0.254643,0.952078,...,0.192697,0.072479,0.296088,0.473088,-0.265873,0.282758,-0.216647,-0.117650,-0.140556,-0.036257
2,2,20.157560,-4.956659,5.808807,3.058060,0.412849,-3.123765,6.337488,-1.362786,12.199962,...,-0.037271,-0.018853,0.019632,-0.043828,0.022645,0.019555,0.000274,0.024939,-0.032378,0.005250
3,3,-2.728951,-3.181694,-0.976422,-0.756793,-1.474728,-2.258582,2.392856,3.796879,-0.553984,...,0.364335,0.456459,-0.276139,0.024653,-0.081194,0.087733,0.094406,0.114546,0.269511,-0.117512
4,4,7.702322,-5.443598,-2.159809,0.679606,-3.023895,-2.976835,9.685654,-5.026649,-4.628518,...,-0.005657,0.126323,0.029598,-0.054593,-0.095143,0.036165,-0.158493,0.081657,0.045822,-0.559805


In [12]:
x.drop(x.columns[[0]], axis=1, inplace=True)
x.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,-0.538296,-5.466138,-0.540832,-1.155411,1.677357,-3.437855,-0.523041,-2.208639,0.969747,0.182594,...,0.255608,0.275158,-0.185373,0.103836,0.040434,-0.059653,-0.189996,0.364697,-0.137938,-0.160443
1,-2.542841,4.907548,1.447379,4.933750,-4.479990,0.505414,4.011756,0.254643,0.952078,1.530630,...,0.192697,0.072479,0.296088,0.473088,-0.265873,0.282758,-0.216647,-0.117650,-0.140556,-0.036257
2,20.157560,-4.956659,5.808807,3.058060,0.412849,-3.123765,6.337488,-1.362786,12.199962,-5.851593,...,-0.037271,-0.018853,0.019632,-0.043828,0.022645,0.019555,0.000274,0.024939,-0.032378,0.005250
3,-2.728951,-3.181694,-0.976422,-0.756793,-1.474728,-2.258582,2.392856,3.796879,-0.553984,0.192194,...,0.364335,0.456459,-0.276139,0.024653,-0.081194,0.087733,0.094406,0.114546,0.269511,-0.117512
4,7.702322,-5.443598,-2.159809,0.679606,-3.023895,-2.976835,9.685654,-5.026649,-4.628518,-4.862649,...,-0.005657,0.126323,0.029598,-0.054593,-0.095143,0.036165,-0.158493,0.081657,0.045822,-0.559805


In [13]:
y=df['Level '] 

**Train Test Split**

In [14]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 200), (524, 200))

In [15]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

2    45.327869
1    29.590164
3    25.081967
Name: Level , dtype: float64


2    45.229008
1    29.961832
3    24.809160
Name: Level , dtype: float64


**Model Building**

In [16]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [17]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test) 

Perceptron F1 score: 0.35757537363246916


**Decision Tree**

In [18]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [19]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [20]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [21]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [22]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [23]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.468852,0.468852,0.578351,0.460504,0.299618,0.299618,0.357759,0.290973


**Random Forest**

In [24]:
clf_rf = RandomForestClassifier(class_weight = "balanced")

In [25]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [26]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [27]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9590    0.9723    0.9656       361
           2     0.9888    0.9602    0.9743       553
           3     0.9464    0.9804    0.9631       306

    accuracy                         0.9689      1220
   macro avg     0.9647    0.9710    0.9677      1220
weighted avg     0.9694    0.9689    0.9689      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2857    0.0764    0.1206       157
           2     0.4474    0.8439    0.5848       237
           3     0.1714    0.0462    0.0727       130

    accuracy                         0.4160       524
   macro avg     0.3015    0.3222    0.2594       524
weighted avg     0.3305    0.4160    0.3187       524



In [28]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.468852,0.468852,0.578351,0.460504,0.299618,0.299618,0.357759,0.290973
1,RandomForest,0.968852,0.968852,0.969357,0.968921,0.416031,0.416031,0.330502,0.318675


**AdaBoost**

In [ ]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5531    0.4183    0.4763       361
           2     0.5714    0.7523    0.6495       553
           3     0.5799    0.4150    0.4838       306

    accuracy                         0.5689      1220
   macro avg     0.5682    0.5285    0.5365      1220
weighted avg     0.5681    0.5689    0.5567      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2696    0.1975    0.2279       157
           2     0.4358    0.5443    0.4841       237
           3     0.2478    0.2154    0.2305       130

    accuracy                         0.3588       524
   macro avg     0.3177    0.3190    0.3141       524
weighted avg     0.3394    0.3588    0.3444       524



In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.577869,0.577869,0.636387,0.558997,0.339695,0.339695,0.327643,0.319543
1,RandomForest,0.969672,0.969672,0.970081,0.969646,0.396947,0.396947,0.328155,0.325998
2,Adaboost,0.568852,0.568852,0.568136,0.556700,0.358779,0.358779,0.339354,0.344401


**Gradient Boosting Classifier**

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9685    0.9363    0.9521       361
           2     0.9291    0.9946    0.9607       553
           3     0.9928    0.9052    0.9470       306

    accuracy                         0.9549      1220
   macro avg     0.9635    0.9454    0.9533      1220
weighted avg     0.9567    0.9549    0.9547      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3109    0.2357    0.2681       157
           2     0.4336    0.6203    0.5104       237
           3     0.2879    0.1462    0.1939       130

    accuracy                         0.3874       524
   macro avg     0.3441    0.3340    0.3241       524
weighted avg     0.3607    0.3874    0.3593       524



In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.577869,0.577869,0.636387,0.558997,0.339695,0.339695,0.327643,0.319543
1,RandomForest,0.969672,0.969672,0.970081,0.969646,0.396947,0.396947,0.328155,0.325998
2,Adaboost,0.568852,0.568852,0.568136,0.556700,0.358779,0.358779,0.339354,0.344401
3,GBM,0.954918,0.954918,0.956717,0.954724,0.387405,0.387405,0.360705,0.359288


**SVM Model**

In [29]:
linear_svm = SVC(kernel='linear', C=1, class_weight = "balanced")

In [30]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [31]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [32]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5553    0.6814    0.6119       361
           2     0.7246    0.5425    0.6205       553
           3     0.5620    0.6667    0.6099       306

    accuracy                         0.6148      1220
   macro avg     0.6140    0.6302    0.6141      1220
weighted avg     0.6337    0.6148    0.6153      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2841    0.3185    0.3003       157
           2     0.4270    0.3333    0.3744       237
           3     0.2454    0.3077    0.2730       130

    accuracy                         0.3225       524
   macro avg     0.3188    0.3198    0.3159       524
weighted avg     0.3391    0.3225    0.3271       524



In [33]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.468852,0.468852,0.578351,0.460504,0.299618,0.299618,0.357759,0.290973
1,RandomForest,0.968852,0.968852,0.969357,0.968921,0.416031,0.416031,0.330502,0.318675
2,SVM,0.614754,0.614754,0.633735,0.615289,0.322519,0.322519,0.339140,0.327055


**KNN**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4684    0.5346    0.4994       361
           2     0.5710    0.7052    0.6311       553
           3     0.5440    0.2222    0.3155       306

    accuracy                         0.5336      1220
   macro avg     0.5278    0.4874    0.4820      1220
weighted avg     0.5339    0.5336    0.5130      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2704    0.3376    0.3003       157
           2     0.4281    0.5021    0.4621       237
           3     0.3000    0.1154    0.1667       130

    accuracy                         0.3569       524
   macro avg     0.3328    0.3184    0.3097       524
weighted avg     0.3491    0.3569    0.3403       524



In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.577869,0.577869,0.636387,0.558997,0.339695,0.339695,0.327643,0.319543
1,RandomForest,0.969672,0.969672,0.970081,0.969646,0.396947,0.396947,0.328155,0.325998
2,Adaboost,0.568852,0.568852,0.568136,0.556700,0.358779,0.358779,0.339354,0.344401
3,GBM,0.954918,0.954918,0.956717,0.954724,0.387405,0.387405,0.360705,0.359288
4,SVM,0.631967,0.631967,0.633660,0.624918,0.362595,0.362595,0.352293,0.355820
5,KNN,0.533607,0.533607,0.533887,0.512954,0.356870,0.356870,0.349053,0.340338


**GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5210    0.4127    0.4606       361
           2     0.5296    0.7107    0.6069       553
           3     0.5833    0.3660    0.4498       306

    accuracy                         0.5361      1220
   macro avg     0.5447    0.4965    0.5058      1220
weighted avg     0.5405    0.5361    0.5242      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2824    0.2357    0.2569       157
           2     0.4361    0.5612    0.4908       237
           3     0.2273    0.1538    0.1835       130

    accuracy                         0.3626       524
   macro avg     0.3153    0.3169    0.3104       524
weighted avg     0.3382    0.3626    0.3445       524



In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.577869,0.577869,0.636387,0.558997,0.339695,0.339695,0.327643,0.319543
1,RandomForest,0.969672,0.969672,0.970081,0.969646,0.396947,0.396947,0.328155,0.325998
2,Adaboost,0.568852,0.568852,0.568136,0.556700,0.358779,0.358779,0.339354,0.344401
3,GBM,0.954918,0.954918,0.956717,0.954724,0.387405,0.387405,0.360705,0.359288
4,SVM,0.631967,0.631967,0.633660,0.624918,0.362595,0.362595,0.352293,0.355820
5,KNN,0.533607,0.533607,0.533887,0.512954,0.356870,0.356870,0.349053,0.340338
6,GNB,0.536066,0.536066,0.540549,0.524224,0.362595,0.362595,0.338238,0.344479
